In [1]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from openai import OpenAI
import pandas as pd
import numpy as np
import torch

import json
import os
import re

from dotenv import load_dotenv
from huggingface_hub import login


load_dotenv()
access_token = os.getenv("HF_TOKEN")
login(token=access_token)

/home/ncoleban/DSC180_B11_Q2/mykernel/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
with open('data/txts/0.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chunks = text.split('\n')

In [3]:
chunk_num = 1 #skips paper id
new_chunks = []
while chunk_num < len(chunks):
    chunk = chunks[chunk_num]
    chunk_num += 1
    if chunk.startswith('\t\t\t') or len(chunk.strip()) == 0:
        continue

    new_chunks.append(chunk)

In [4]:
# model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-7B-instruct")

# simple_prompt = "What, if any, is the passivating molecule tested, and what is the corresponding PCE, VOC, and stability test data (efficency retained over time, temperature, test type)"

# query = model.encode(simple_prompt, convert_to_tensor=True)

# cosine_similarities = []
# for chunk in new_chunks:
#     text_embeddings = model.encode(chunk, convert_to_tensor=True)
#     cosine_similarities.append(util.pytorch_cos_sim(query, text_embeddings).item())

# cos_mean = np.mean(cosine_similarities)

# classified_chunks = []
# for i, value in enumerate(cosine_similarities):
#     if value >= cos_mean:
#         classified_chunks.append(new_chunks[i])

In [5]:
system_prompt = (
    "You are a scientific assistant and your task is to extract certain information from text. "
    "We are in a scientific environment. You MUST be critical of the units of the variables. "
    "Do not leave information behind. "
    "Only extract the variables that were developed in this study. You must omit the ones extracted from the bibliography"
)

In [6]:
json_schema = {
    "perovskite_composition": {"type:": str},
    "electron_transport_layer": {"type": str},
    "hole_transport_layer": {"type": str},
    "structure_pin_nip": {"type": str},
    "test_1": {
        "stability_type": {"type": str},
        "passivating_molecule": {"type": str},
        "humidity": {"type": int},
        "temperature": {"type": int},
        "time": {"type": int},
        "control_pce": {"type": float},
        "treated_pce": {"type": float},
        "control_voc": {"type": float},
        "treated_voc": {"type": float},
        "efficiency_control": {"type": float},
        "efficiency_tret": {"type": float}
    }
}

In [7]:
simple_prompt = """Extract only the variables detailed below from the provided text. Then join them with the data from previous chunks.
It is likely that a lot of this data is not present in the chunk provided. Only extract the data points that are present in the chunk.
To provide the answer with the data, follow the schema outlined in the JSON provided:

{json_schema}

Some important notes about the variables:

-`passivating_molecule`: Full name of the passivating molecule tested.
-`treated_pce`: PCE (Power Conversion Efficiency) of the perovskite with the passivating treatment.
-`control_pce`: PCE of the control perovskite.
-`treated_voc`: VOC of the perovskite with the passivating treatment.
-`control_voc`: VOC of the control perovskite. 
-`time`: Stability test length in hours
-`efficiency_tret`: Percent efficiency retained after stability test length for perovskite with passivating treatment.
-`efficiency_control`: Percent efficiency retained after stability test length for control perovskite.

Text to extract from:

{chunk}

Add the newly extracted data to the one from previous chunks that is the following:

{memory}

Never leave the information from previous chunks behind.

Begin extracting!
"""

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=getattr(torch, "float16"),
    bnb_4bit_use_double_quant=False
)

In [9]:
# client = OpenAI(api_key=os.getenv("DEEPINFRA_TOKEN"), base_url="https://api.deepinfra.com/v1/openai")
# client = OpenAI(api_key=os.getenv("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com")
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_name = "models/DeepSeek-R1-PSC-Extractor-8B"
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
tokenizer_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer.model_max_length = 20000
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=None,
    top_p=None,
    do_sample=False,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.82s/it]


In [10]:
summary = ""
i = 0 
for chunk in new_chunks:
    if i == 2:
        i += 1
        continue
    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        { "role": "user", "content": simple_prompt.format(
            json_schema=json_schema, 
            chunk=chunk, 
            memory=summary,
            )
        }
    ]
    # response = client.chat.completions.create(
    #     model="deepseek-chat",
    #     messages=messages,
    #     stream=False,
    #     max_tokens=8192
    # )
    # summary = response.choices[0].message.content
    summary = pipe(messages, max_new_tokens=1024)[0]["generated_text"][-1]['content']
    i += 1

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


KeyboardInterrupt: 

In [11]:
i

36

In [12]:
cleaned_string = summary.strip().strip('```json').strip()
json.loads(cleaned_string)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
summary

'Alright, I\'m looking at the text provided and the instructions to extract specific information. The text mentions a statistic data figure (S14) with different concentrations of 4Cl-BZS additive. However, the user has already provided data from a previous chunk, which includes details about the perovskite composition, transport layers, and stability tests. \n\nSince the task is to only extract variables developed in this study and not to include data from previous chunks, I need to focus on the current text. The text doesn\'t provide any new information about the variables listed in the schema. It only refers to a figure with statistical data, which isn\'t directly related to the variables we\'re supposed to extract. \n\nTherefore, I won\'t add any new data to the existing information from the previous chunk. The response remains the same as the one provided for the previous chunk.\n</think>\n\n{"perovskite_composition": "formamidinium lead iodide and formamidinium lead bromide", "ele

### Evaluation

In [ ]:
training_data = pd.read_csv('data/training_data.csv')
paper_0_json = str(training_data[training_data["id"] == 0]["output"])


In [ ]:
str(messages[1])

"{'role': 'user', 'content': 'Extract only the variables detailed below from the provided text. Then join them with the data from previous chunks.\\nIt is likely that a lot of this data is not present in the chunk provided. Only extract the data points that are present in the chunk.\\nTo provide the answer with the data, do it using a schema similar to the following:\\n\\n- Full name of the passivating molecule tested.\\n- PCE (Power Conversion Efficiency) of the perovskite with the passivating treatment.\\n- PCE of the control perovskite.\\n- VOC of the perovskite with the passivating treatment.\\n- VOC of the control perovskite. \\n- Stability test type\\n- Stability test length\\n- Percent efficiency retained after stability test length for perovskite with passivating treatment.\\n- Percent efficiency retained after stability test length for control perovskite.\\n\\nText to extract from:\\n\\n Table S5 . S5 Summary of operational stability of reported inverted PSCs following the ISO

In [ ]:
outputs = {}
outputs['LLAMA-70B-IT'] = summary

In [ ]:
outputs

{'LLAMA-70B-IT': "After carefully reviewing the provided text, I extracted the following new data points that match the specified variables:\n\n- Full name of the passivating molecule tested: Not mentioned\n- PCE (Power Conversion Efficiency) of the perovskite with the passivating treatment: Not mentioned\n- PCE of the control perovskite: Not mentioned\n- VOC of the perovskite with the passivating treatment: Not mentioned\n- VOC of the control perovskite: Not mentioned\n- Stability test type: Not mentioned\n- Stability test length: Not mentioned\n- Percent efficiency retained after stability test length for perovskite with passivating treatment: Not mentioned\n- Percent efficiency retained after stability test length for control perovskite: Not mentioned\n\nI will add these new data points to the existing data sheet, ensuring that I don't leave any information behind. Here is the updated data sheet:\n\n- Full name of the passivating molecule tested: cyclohexylmethylammonium iodide (CMA